In [30]:
from moviepy.editor import *
import asyncio
import nest_asyncio
from queue import Queue
from threading import Thread
nest_asyncio.apply()

# 1. Loading appropriate json files

- Change the dir inside the open() to read different json datasets
- Change the 'limit' var to limit number of data downloaded (default: 99999)



In [42]:
# limits number of files to download as not to chew through SSD write cycle
limit = 99999
#limit = 100

limitedDataset = []

import json

with open('dataset-ref/MSASL_train.json') as f:
    d = json.load(f)
    
    for i in range(limit):
        
        limitedDataset.append([d[i]['clean_text'], d[i]['url'], d[i]['start_time'], d[i]['end_time']])
        print(d[i])
        


{'org_text': 'match [light-a-MATCH]', 'clean_text': 'match', 'start_time': 0.0, 'signer_id': 0, 'signer': 0, 'start': 0, 'end': 83, 'file': 'match light-a-MATCH', 'label': 830, 'height': 360.0, 'fps': 30.0, 'end_time': 2.767, 'url': 'https://www.youtube.com/watch?v=C37R_Ix8-qs', 'text': 'match', 'box': [0.05754461884498596, 0.21637457609176636, 1.0, 0.7300844192504883], 'width': 640.0}
{'org_text': 'FAIL', 'clean_text': 'fail', 'start_time': 0.0, 'signer_id': 0, 'signer': -1, 'start': 0, 'end': 74, 'file': 'FAIL', 'label': 542, 'height': 360.0, 'fps': 25.0, 'end_time': 2.96, 'url': 'https://www.youtube.com/watch?v=PIsUJl8BN_I', 'text': 'fail', 'box': [0.0657794177532196, 0.16717177629470825, 0.9392627477645874, 0.9187960028648376], 'width': 480.0}
{'org_text': 'laugh', 'clean_text': 'laugh', 'start_time': 0.0, 'signer_id': 4, 'signer': 26, 'start': 0, 'end': 31, 'file': 'SignSchool Laugh with Legs 2', 'label': 312, 'height': 360.0, 'fps': 29.97, 'end_time': 1.034, 'url': 'www.youtube.c

# 2. Set folder paths

- Set 'folder_path' to your base path
- You can set the dataset_path or trimmed_dataset_path to your desired folder, or leave it as default

### dataset_path
- Stores raw, full length video downloaded from youtube, as some gestures are a snippet of the long videos.
- Stores the file names in slugs.

### trimmed_dataset
- Once the raw videos are downloaded, the video will be fed into moviepy to be trimmed according to the specified time in the json files.
- The resulting videos resides here.



In [44]:
import os
import glob

folder_path = 'C:/your/path/to/folder/'
# folder_path = 'D:/University/Y3/AITT/ActionDetectionforSignLanguage/'
file_extension = 'mp4'

# output paths
dataset_path = '/dataset/'
trimmed_dataset_path = '/trimmed-dataset/'

# Get a list of all mp4 files in the folder
files = os.listdir(folder_path + dataset_path)

filenames = []

async def loadFilenames():
    # Print the list of files
    for file in files:
        filenames.append(os.path.splitext(file)[0])
        # print(filenames)
        # print(len(filenames))
        
asyncio.run(loadFilenames())
    

In [33]:
from urllib.parse import urlparse, parse_qs

fileLookup = {}
toDownload = {}

# for i in limitedDataset:
#     for j in filenames:
#         if any(string in filenames):
#             print(string + "is in")
#         else:
#             print(string + "is not in")


def query(url):
    try:
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        # print(query_params['v'])
        # toDownload.append((query_params['v']))
        return query_params['v']

    except Exception as e:
        print(e)
        url = 'https://' + url
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        return query_params['v']


async def getVideoUrl():
    for subarray in limitedDataset:
        queriedLink = query(subarray[1])
        # print('queriedLink ' + queriedLink[0])
        if all([queriedLink[0] not in filenames]):
            toDownload[queriedLink[0]] = []
            print(toDownload)
            # toDownload.append(subarray[1])


async def getTimeStamps():
    await getVideoUrl()
    await loadFilenames()
    
    for subarray in limitedDataset:
        queriedLink = query(subarray[1])
        if all([queriedLink[0] not in filenames]):
            print(queriedLink[0])
            toDownload.get(queriedLink[0]).append(
                [subarray[0], subarray[2], subarray[3]])
            
async def getAllVideoUrl():
    for subarray in limitedDataset:
        queriedLink = query(subarray[1])
        # print('queriedLink ' + queriedLink[0])
        if all([queriedLink[0] in filenames]):
            fileLookup[queriedLink[0]] = []
            print(toDownload)
            # toDownload.append(subarray[1])


async def getAllTimeStamps():
    await loadFilenames()
    await getAllVideoUrl()
    
    for subarray in limitedDataset:
        queriedLink = query(subarray[1])
        if all([queriedLink[0] in filenames]):
            fileLookup.get(queriedLink[0]).append(
                [subarray[0], subarray[2], subarray[3]])
            
            
    print(len(fileLookup))


asyncio.run(getTimeStamps())
asyncio.run(getAllTimeStamps())

# create a dictionary to store video url, then store id, start and end


{'9FdHlMOnVjg': []}
{'9FdHlMOnVjg': [], '1AyT77LqJzQ': []}
{'9FdHlMOnVjg': [], '1AyT77LqJzQ': []}
{'9FdHlMOnVjg': [], '1AyT77LqJzQ': [], 'cJOyCgIKyeA': []}
{'9FdHlMOnVjg': [], '1AyT77LqJzQ': [], 'cJOyCgIKyeA': [], 'zqfcp6NRKpA': []}
{'9FdHlMOnVjg': [], '1AyT77LqJzQ': [], 'cJOyCgIKyeA': [], 'zqfcp6NRKpA': [], '7y5Ye-2-ZBs': []}
{'9FdHlMOnVjg': [], '1AyT77LqJzQ': [], 'cJOyCgIKyeA': [], 'zqfcp6NRKpA': [], '7y5Ye-2-ZBs': [], 'rYW7VFPi9_w': []}
{'9FdHlMOnVjg': [], '1AyT77LqJzQ': [], 'cJOyCgIKyeA': [], 'zqfcp6NRKpA': [], '7y5Ye-2-ZBs': [], 'rYW7VFPi9_w': [], '0Beq_NIDj2c': []}
{'9FdHlMOnVjg': [], '1AyT77LqJzQ': [], 'cJOyCgIKyeA': [], 'zqfcp6NRKpA': [], '7y5Ye-2-ZBs': [], 'rYW7VFPi9_w': [], '0Beq_NIDj2c': []}
{'9FdHlMOnVjg': [], '1AyT77LqJzQ': [], 'cJOyCgIKyeA': [], 'zqfcp6NRKpA': [], '7y5Ye-2-ZBs': [], 'rYW7VFPi9_w': [], '0Beq_NIDj2c': [], 'AoQAPgEUIAs': []}
{'9FdHlMOnVjg': [], '1AyT77LqJzQ': [], 'cJOyCgIKyeA': [], 'zqfcp6NRKpA': [], '7y5Ye-2-ZBs': [], 'rYW7VFPi9_w': [], '0Beq_NIDj2c': [], '

# 3. Video Downloader

### IMPORTANT
- Set the 'num_worker_threads' accroding to the CPU your machine has. 

- Failure to set the appropriate thread might result in crashing.

- If you're unsure how many threads your CPU has, go with
num_worker_threads = 4

- I was using R7 4800H with 16 supported threads, hence
num_worker_threads = 16

### Function

- Downloads the video using pytube (with multithreading support, although I can't verify if it works or not, do let me know your feedback)
- Some videos were privated. Privated or missing videos are skipped.

In [34]:
from pytube import YouTube
from pytube.exceptions import VideoPrivate
from pytube.exceptions import VideoUnavailable
from pytube.cli import on_progress
from pytube import YouTube
from queue import Queue
from threading import Thread

# IMPORTANT: Set the number of threads according to the CPU your machine has
num_worker_threads = 4
# num_worker_threads = 16

def downloadVideo(file):
    i = file
    try:
        path = dataset_path
        url = 'https://www.youtube.com/watch?v=' + i
        print(url)
        yt = YouTube(url)
        
        stream = yt.streams.filter(file_extension='mp4').first()
        

    except VideoUnavailable as e:
        print(e)
        print(i + ' is private')
        
    except Exception as e:
        print(e)

    else:
        if(stream != None):
            stream.download(filename=i + ".mp4", output_path=path)
            print('downloading')

# Multithreading
def worker(q):
    while True:
        item = q.get()
        if item is None:
            break
        # Your code to process the item here
        q.task_done()


q = Queue()
threads = []
for i in range(num_worker_threads):
    t = Thread(target=worker, args=(q,))
    t.start()
    threads.append(t)

# Your code to add items to the queue here
for item in toDownload:
    q.put(downloadVideo(item))
    q.task_done()

# Block until all tasks are done
q.join()

# Stop workers
for i in range(num_worker_threads):
    q.put(None)
for t in threads:
    t.join()


https://www.youtube.com/watch?v=9FdHlMOnVjg
9FdHlMOnVjg is a private video
9FdHlMOnVjg is private
https://www.youtube.com/watch?v=1AyT77LqJzQ
1AyT77LqJzQ is unavailable
1AyT77LqJzQ is private
https://www.youtube.com/watch?v=cJOyCgIKyeA
cJOyCgIKyeA is unavailable
cJOyCgIKyeA is private
https://www.youtube.com/watch?v=zqfcp6NRKpA
zqfcp6NRKpA is a private video
zqfcp6NRKpA is private
https://www.youtube.com/watch?v=7y5Ye-2-ZBs
7y5Ye-2-ZBs is a private video
7y5Ye-2-ZBs is private
https://www.youtube.com/watch?v=rYW7VFPi9_w
rYW7VFPi9_w is a private video
rYW7VFPi9_w is private
https://www.youtube.com/watch?v=0Beq_NIDj2c
0Beq_NIDj2c is unavailable
0Beq_NIDj2c is private
https://www.youtube.com/watch?v=AoQAPgEUIAs
AoQAPgEUIAs is unavailable
AoQAPgEUIAs is private
https://www.youtube.com/watch?v=25BrmaKbzYo
25BrmaKbzYo is a private video
25BrmaKbzYo is private


# 4. Video Trimming

- Refers to json file for timestamps via youtube slug, then trims the raw videos in 'dataset_path' using moviepy. Output will be saved in 'trimmed_dataset_path'
- Uses multithreading, with the number of workers set above previously.

#### Voila!


In [46]:



# Get a list of all mp4 files in the folder
files = os.listdir(folder_path + dataset_path)

videonames = []
availableVideos = []
# trimmed_folder_path = 'D:/University/Y3/AITT/ActionDetectionforSignLanguage/trimmed-dataset'
file_extension = 'mp4'

async def getDatasetNames():
# Print the list of dataset files
    for file in files:
        videonames.append(os.path.splitext(file)[0])

# Get a list of all mp4 files in trimmed_dataset
trimmedFiles = os.listdir(folder_path + trimmed_dataset_path)
        
trimmedVideoNames = []

async def getTrimmedDatasetNames():
    # Print the list of files
    for file in trimmedFiles:
        trimmedVideoNames.append(os.path.splitext(file)[0])
        
def trimVid(name, segments):
    print(name)
    getTrimmedDatasetNames()
    for i in segments:
        outputName = i[0]
        startTime = i[1]
        endTime = i[2]
        video = VideoFileClip(dataset_path + name +".mp4").subclip(startTime, endTime)
        # check if file exists in trimmed-dataset
        if any([outputName in trimmedVideoNames]):
            print('file exists')
            print('I should be merging ' + outputName)
            video_2 = VideoFileClip(trimmed_dataset_path + outputName +".mp4")
            finalvideo = concatenate_videoclips([video, video_2])
            finalvideo.write_videofile(trimmed_dataset_path + outputName + ".mp4", fps=25)
        else:
            video.write_videofile(trimmed_dataset_path + outputName + ".mp4", fps=25)

async def vidController():
    await getDatasetNames()
    

asyncio.run(vidController())

# Multithreaded
def worker(q):
    while True:
        item = q.get()
        if item is None:
            break
        # Your code to process the item here
        q.task_done()

q = Queue()
threads = []
for i in range(num_worker_threads):
    t = Thread(target=worker, args=(q,))
    t.start()
    threads.append(t)

# Your code to add items to the queue here
for name in videonames:
    q.put(trimVid(name, fileLookup[name]))
    q.task_done()

# Block until all tasks are done
q.join()

# Stop workers
for i in range(num_worker_threads):
    q.put(None)
for t in threads:
    t.join()



5WevUel4BPk
Moviepy - Building video trimmed-dataset/there.mp4.
MoviePy - Writing audio in thereTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video trimmed-dataset/there.mp4



Moviepy - Done !
Moviepy - video ready trimmed-dataset/there.mp4
7G6B6kaMjb4
Moviepy - Building video trimmed-dataset/give.mp4.
MoviePy - Writing audio in giveTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video trimmed-dataset/give.mp4



Moviepy - Done !
Moviepy - video ready trimmed-dataset/give.mp4
Bn7j1v1ksl0
Moviepy - Building video trimmed-dataset/meat.mp4.
MoviePy - Writing audio in meatTEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready trimmed-dataset/think.mp4
_HOx2QkkTsg
Moviepy - Building video trimmed-dataset/teacher.mp4.
MoviePy - Writing audio in teacherTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video trimmed-dataset/teacher.mp4



Moviepy - Done !
Moviepy - video ready trimmed-dataset/teacher.mp4
Moviepy - Building video trimmed-dataset/teacher.mp4.
MoviePy - Writing audio in teacherTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video trimmed-dataset/teacher.mp4



Moviepy - Done !
Moviepy - video ready trimmed-dataset/teacher.mp4
